In [18]:
import torch
import random
import pandas as pd
import numpy as np
import main_forecasting as mf

In [19]:
# seed 고정
random_seed = 42

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [20]:
# Case 1. model = lstm
config1 = {
    "model": 'lstm',
    "training": True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
    "best_model_path": './ckpt/lstm.pt',  # 학습 완료 모델 저장 경로
    "parameter": {
        "input_size" : 1,  # 데이터 변수 개수, int
        "window_size" : 48,  # input sequence의 길이, int
        "forecast_step" : 24,  # 예측할 미래 시점의 길이, int
        "num_layers" : 2,  # recurrent layers의 수, int(default: 2, 범위: 1 이상)
        "hidden_size" : 64,  # hidden state의 차원, int(default: 64, 범위: 1 이상)
        "dropout" : 0.1,  # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
        "bidirectional" : True,  # 모델의 양방향성 여부, bool(default: True)
        "num_epochs" : 150,  # 학습 epoch 횟수, int(default: 150, 범위: 1 이상)
        "batch_size" : 64,  # batch 크기, int(default: 64, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
        "lr" : 0.0001,  # learning rate, float(default: 0.0001, 범위: 0.1 이하)
        "device" : 'cuda'  # 학습 환경, (default: 'cuda', ['cuda', 'cpu'] 중 선택)
    }
}

# Case 2. model = gru
config2 = {
    "model": 'gru',
    "training": True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
    "best_model_path": './ckpt/gru.pt',  # 학습 완료 모델 저장 경로
    "parameter": {
        "input_size" : 1,  # 데이터 변수 개수, int
        "window_size" : 48,  # input sequence의 길이, int
        "forecast_step" : 24,  # 예측할 미래 시점의 길이, int
        "num_layers" : 2,  # recurrent layers의 수, int(default: 2, 범위: 1 이상)
        "hidden_size" : 64,  # hidden state의 차원, int(default: 64, 범위: 1 이상)
        "dropout" : 0.1,  # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
        "bidirectional" : True,  # 모델의 양방향성 여부, bool(default: True)
        "num_epochs" : 150,  # 학습 epoch 횟수, int(default: 150, 범위: 1 이상)
        "batch_size" : 64,  # batch 크기, int(default: 64, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
        "lr" : 0.0001,  # learning rate, float(default: 0.0001, 범위: 0.1 이하)
        "device" : 'cuda'  # 학습 환경, (default: 'cuda', ['cuda', 'cpu'] 중 선택)
    }
}

# Case 3. model = informer
config3 = {
    "model": 'informer',
    "training": True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
    "best_model_path": './ckpt/informer.pt',  # 학습 완료 모델 저장 경로
    "parameter": {
        "input_size" : 1,  # 데이터 변수 개수, int
        "window_size" : 48,  # input sequence의 길이, int
        "forecast_step" : 24,  # 예측할 미래 시점의 길이, int
        "label_len" : 12,  # Decoder의 start token 길이, int(default: 12)
        "d_model" : 512,  # 모델의 hidden dimension, int(default: 512)
        "e_layers" : 2,  # encoder layer 수, int(default: 2)
        "d_layers" : 1,  # decoder layer 수, int(default: 1)
        "d_ff" : 2048,  # fully connected layer의 hidden dimension, int(default: 2048)
        "num_epochs" : 150,  # 학습 epoch 횟수, int(default: 150, 범위: 1 이상)
        "batch_size" : 64,  # batch 크기, int(default: 64, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
        "lr" : 0.0001,  # learning rate, float(default: 0.0001, 범위: 0.1 이하)
        "device" : 'cuda',  # 학습 환경, (default: 'cuda', ['cuda', 'cpu'] 중 선택)
        "factor" : 5, # 모델의 ProbSparse Attention factor, int(default: 5)
        "dropout" : 0.05, # dropout ratio, int(default: 0.05)
        "attn" : 'prob', # 모델의 attention 계산 방식, (default: 'prob', ['prob', 'full'] 중 선택)
        "n_heads" : 8, # multi-head attention head 수, int(default: 8)
        "embed" : 'timeF', # time features encoding 방식, (default: 'timeF', ['timeF', 'fixed', 'learned'] 중 선택)
        "lradj" : 'type1' # learning rate 조정 방식, (default: 'type1', ['type1', 'type2'] 중 선택)
    }
}

# Case 4. model = scinet
config4 = {
    "model": 'scinet',
    "training": True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
    "best_model_path": './ckpt/scinet.pt',  # 학습 완료 모델 저장 경로
    "parameter": {
        "input_size" : 1,  # 데이터 변수 개수, int
        "window_size" : 48,  # input sequence의 길이, int
        "forecast_step" : 24,  # 예측할 미래 시점의 길이, int
        "levels" : 2, # Tree의 depth, int(default: 2, 범위: input sequence의 로그 값 이하, 2~4 설정 권장)
        "stacks" : 1, # SCINet 구조를 쌓는 횟수, int(default: 1, 범위: 3 이하)
        "num_epochs" : 150,  # 학습 epoch 횟수, int(default: 150, 범위: 1 이상)
        "batch_size" : 64,  # batch 크기, int(default: 64, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
        "lr" : 0.0001,  # learning rate, float(default: 0.0001, 범위: 0.1 이하)
        "device" : 'cuda'  # 학습 환경, (default: 'cuda', ['cuda', 'cpu'] 중 선택)
    }
}

In [21]:
# Dataset
dataset_dir = {
    "train": './data/train_data.csv',
    "test": './data/test_data.csv'
}

# train/test 데이터 불러오기 (csv 형태)
# shape=(# time steps, )
train_data = pd.read_csv(dataset_dir["train"])
train_data = train_data["Appliances"].values

test_data = pd.read_csv(dataset_dir["test"])
test_date = test_data["date"].values
test_data = test_data["Appliances"].values

In [22]:
# Case 1. model = lstm
config = config1
data_forecast = mf.Forecasting(config, train_data, test_data, test_date)
init_model = data_forecast.build_model()  # 모델 구축

if config["training"]:
    best_model = data_forecast.train_model(init_model)  # 모델 학습
    data_forecast.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

pred, mse, mae = data_forecast.pred_data(init_model, best_model_path=config["best_model_path"])  # 예측
print(f'MSE : {mse} , MAE : {mae}')

Start training model

Epoch 1/150
----------
train Loss: 0.9989 RMSE: 0.9995
val Loss: 0.9087 RMSE: 0.9533

Epoch 2/150
----------
train Loss: 0.9976 RMSE: 0.9988
val Loss: 0.9075 RMSE: 0.9526

Epoch 3/150
----------
train Loss: 0.9964 RMSE: 0.9982
val Loss: 0.9063 RMSE: 0.9520

Epoch 4/150
----------
train Loss: 0.9952 RMSE: 0.9976
val Loss: 0.9049 RMSE: 0.9513

Epoch 5/150
----------
train Loss: 0.9941 RMSE: 0.9970
val Loss: 0.9036 RMSE: 0.9506

Epoch 6/150
----------
train Loss: 0.9928 RMSE: 0.9964
val Loss: 0.9021 RMSE: 0.9498

Epoch 7/150
----------
train Loss: 0.9914 RMSE: 0.9957
val Loss: 0.9004 RMSE: 0.9489

Epoch 8/150
----------
train Loss: 0.9900 RMSE: 0.9950
val Loss: 0.8984 RMSE: 0.9478

Epoch 9/150
----------
train Loss: 0.9882 RMSE: 0.9941
val Loss: 0.8958 RMSE: 0.9465

Epoch 10/150
----------
train Loss: 0.9861 RMSE: 0.9930
val Loss: 0.8927 RMSE: 0.9448

Epoch 11/150
----------
train Loss: 0.9835 RMSE: 0.9917
val Loss: 0.8890 RMSE: 0.9429

Epoch 12/150
----------
train 

In [23]:
# Case 2. model = gru
config = config2
data_forecast = mf.Forecasting(config, train_data, test_data, test_date)
init_model = data_forecast.build_model()  # 모델 구축

if config["training"]:
    best_model = data_forecast.train_model(init_model)  # 모델 학습
    data_forecast.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

pred, mse, mae = data_forecast.pred_data(init_model, best_model_path=config["best_model_path"]) # 예측
print(f'MSE : {mse} , MAE : {mae}')

Start training model

Epoch 1/150
----------
train Loss: 1.0045 RMSE: 1.0023
val Loss: 0.9157 RMSE: 0.9569

Epoch 2/150
----------
train Loss: 1.0001 RMSE: 1.0001
val Loss: 0.9103 RMSE: 0.9541

Epoch 3/150
----------
train Loss: 0.9957 RMSE: 0.9978
val Loss: 0.9053 RMSE: 0.9515

Epoch 4/150
----------
train Loss: 0.9917 RMSE: 0.9959
val Loss: 0.9001 RMSE: 0.9487

Epoch 5/150
----------
train Loss: 0.9878 RMSE: 0.9939
val Loss: 0.8947 RMSE: 0.9459

Epoch 6/150
----------
train Loss: 0.9835 RMSE: 0.9917
val Loss: 0.8891 RMSE: 0.9429

Epoch 7/150
----------
train Loss: 0.9793 RMSE: 0.9896
val Loss: 0.8832 RMSE: 0.9398

Epoch 8/150
----------
train Loss: 0.9749 RMSE: 0.9873
val Loss: 0.8773 RMSE: 0.9367

Epoch 9/150
----------
train Loss: 0.9706 RMSE: 0.9852
val Loss: 0.8709 RMSE: 0.9332

Epoch 10/150
----------
train Loss: 0.9656 RMSE: 0.9827
val Loss: 0.8641 RMSE: 0.9296

Epoch 11/150
----------
train Loss: 0.9605 RMSE: 0.9801
val Loss: 0.8563 RMSE: 0.9253

Epoch 12/150
----------
train 

In [ ]:
# Case 3. model = informer
config = config3
data_forecast = mf.Forecasting(config, train_data, test_data, test_date)
init_model = data_forecast.build_model()  # 모델 구축

if config["training"]:
    best_model = data_forecast.train_model(init_model)  # 모델 학습
    data_forecast.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

pred, mse, mae = data_forecast.pred_data(init_model, best_model_path=config["best_model_path"])  # 예측

In [ ]:
# Case 4. model = scinet
config = config4
data_forecast = mf.Forecasting(config, train_data, test_data, test_date)
init_model = data_forecast.build_model()  # 모델 구축

if config["training"]:
    best_model = data_forecast.train_model(init_model)  # 모델 학습
    data_forecast.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

pred, mse, mae = data_forecast.pred_data(init_model, best_model_path=config["best_model_path"])  # 예측